# Klassifikation der Anfänge von Kurzgeschichten nach ihrem Informationsgrad

In [1]:
pip install openai


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

# Loading Data

In [4]:
file_path = 'Kurzgeschichten_Anfaenge_DHD_2025.xlsx'

In [5]:
df = pd.read_excel(file_path)

In [6]:
df.head()

,Autor,Titel,Absatz,Eingangssatz,Person,Gegenstand,Ort,Zeit,Score,Klassifikation
0,Böll,Der Dieb,"Am meisten war ich darüber erstaunt, wie schne...","Am meisten war ich darüber erstaunt, wie schne...",1,1,0,0,2,sehr wenig Information
1,Böll,Aus der »Vorzeit«,Nachdem die Kompanie vom Kasernenhof abgerückt...,Nachdem die Kompanie vom Kasernenhof abgerückt...,1,2,2,0,5,wenig Information
2,Böll,Im Käfig,Irgendeiner stand am Zaun und blickte grübelnd...,Irgendeiner stand am Zaun und blickte grübelnd...,1,2,2,0,5,wenig Information
3,Böll,In guter Hut,"Als ich wach wurde, wollte ich es zuerst gar n...","Als ich wach wurde, wollte ich es zuerst gar n...",1,2,0,0,3,sehr wenig Information
4,Böll,Todesursache: Hakennase,Als der Leutnant Hegemüller in sein Quartier z...,Als der Leutnant Hegemüller in sein Quartier z...,2,2,2,0,6,wenig Information


In [7]:
df.Klassifikation.value_counts()

Klassifikation
sehr wenig Information    25
wenig Information         22
viel Information           8
Name: count, dtype: int64

## gpt-4 SetUp

The following lines of code and prompts are based on the hints and instructions from the aformentioned course: https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines

In [8]:
import os
import openai

openai.api_key = os.getenv('MY_OPENAI')


In [9]:
def get_completion(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# Evaluating

In [10]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

In [11]:
results = []
for text in df.Eingangssatz:
    snippet = text
    
    prompt = f"""
    #Role
    You are a literary scholar
    
    #Instruction
    Your task is to recognize points of indeterminacy in German-language narrative texts given to you. The opening of a literary narrative has a point of indeterminacy precisely when it is characterized by a lack of information regarding the central character, the subject, the place or the time of the plot.
    The degree of lack of information is determined using the following scorin system: The starting point is information about the character, subject, place and time of the narrative:
    If one or more of these dimensions are represented by pronouns or proper nouns, 1 point is awarded for each.
    If they are represented by definite noun phrases with determiners, 2 points are awarded for each.
    If they are represented by indefinite noun phrases, 3 points are awarded for each.
    If the information on the character, subject, time and place of the story uses a word type other than those mentioned above, no points will be awarded!
    Points can only be awarded once for each of the 4 categories. This means that if there are two information carriers for the location, points are only awarded once for these!

    Give first an analysis, calculate the score and then select one of the following German labels to classify the degree of the lack of information regarding the central character, the subject, the place or the time of the plot in the given text snippet:
        If the total score of the sentence is between 10 and 12 points – Label: sehr viel Information
        If the total score of the sentence is between 7 and 9 points - Label: viel Information
        If the total score of the sentence is between 4 and 6 points - Label: wenig Information
        If the total score of the sentence is between 0 and 3 points - Label: sehr wenig Information
        
    
    #Output
    Use the following output format:
    Analysis: str
    Label: str
    
    #Examples
    Snippet: Seit einiger Zeit findet in dem Atelier neben meiner Wohnung ein rauschendes Fest statt.
    Analysis: In the sentence there is a vague indication of time in a determinate definite noun phrase ("seit einiger Zeit") - 2 points -, an equally vague indication of place in a determinate definite noun phrase ("n dem Atelier neben meiner Wohnung") - 2 points - and an indication of the object of the action in a definite noun phrase ("ein rauschendes Fest") - 3 points -, but no indication of the person acting. The total number of points is 2 + 2 + 3 = 7 points, which is why the sentence is labeled "viel Information"
    Label: viel Information
    
    Snippet: Es hätte mir nichts auszumachen brauchen
    Analysis: In the sentence there is only a vague indication of a person with a personal pronoun ("mir") - 1 point - and an equally vague indication of the subject in the personal pronoun ("Es") - 1 point. Based on the total score of 1 + 1 = 2, the sentence is assigned the label "sehr wenig Information".
    Label: sehr wenig Information
    
    #
    Analyze the following German text: {snippet}
    Analysis: 
    Label:
    """
    
    result = get_completion(prompt)
    print(result)
    results.append(result)

Snippet: Am meisten war ich darüber erstaunt, wie schnell und einfach es ging.

Analysis: In the sentence, there is a vague indication of a person with a personal pronoun ("ich") - 1 point - and an equally vague indication of the subject in the personal pronoun ("es") - 1 point. There is no clear indication of place or time. Based on the total score of 1 + 1 = 2, the sentence is assigned the label "sehr wenig Information".

Label: sehr wenig Information
Snippet: Nachdem die Kompanie vom Kasernenhof abgerückt war, eilte Renatus noch einmal auf die Stube, um, wie es die Vorschrift zu seinem Dienst verlangte, den weißen Drillich anzuziehen.

Analysis: In the sentence, there is a definite noun phrase indicating the character ("Renatus") - 2 points. The subject of the action is also indicated by a definite noun phrase ("die Kompanie") - 2 points. The place is indicated by a definite noun phrase ("vom Kasernenhof" and "auf die Stube") - 2 points. The time is indicated by a definite noun phra

In [12]:
results_1 = []
for text in results:
    if "stark unbestimmt" in text:
        results_1.append("stark unbestimmt")
    elif "wenig unbestimmt" in text:
        results_1.append("wenig unbestimmt")
    elif "nicht unbestimmt" in text:
        results_1.append("nicht unbestimmt")
    elif "unbestimmt" in text:
        results_1.append("unbestimmt")
    else:
        results_1.append("NaN")

In [13]:
results_1 = []
for text in results:
    if "sehr viel Information" in text:
        results_1.append("sehr viel Information")
    elif "viel Information" in text:
        results_1.append("viel Information")
    elif "sehr wenig Information" in text:
        results_1.append("sehr wenig Information")
    elif "wenig Information" in text:
        results_1.append("wenig Information")
    else:
        results_1.append("NaN")

In [14]:
results_se = pd.Series(results_1)

In [15]:
predictions = results_se
ground_truth = df.Klassifikation

In [16]:
pd.DataFrame(zip(ground_truth, predictions))

,0,1
0,sehr wenig Information,sehr wenig Information
1,wenig Information,viel Information
2,wenig Information,viel Information
3,sehr wenig Information,sehr wenig Information
4,wenig Information,wenig Information
5,wenig Information,wenig Information
6,wenig Information,wenig Information
7,wenig Information,wenig Information
8,sehr wenig Information,wenig Information
9,sehr wenig Information,wenig Information


In [17]:
pd.DataFrame(list(zip([f1_score(ground_truth, predictions, average="weighted")],
                      [recall_score(ground_truth, predictions,  average="weighted")],
                      [precision_score(ground_truth, predictions, average="weighted")],
                      [accuracy_score(ground_truth, predictions,)])),
                      columns = ["F1", "Recall", "Precision", "Accuracy"])

,F1,Recall,Precision,Accuracy
0,0.634848,0.636364,0.695238,0.636364


In [18]:
print(classification_report(ground_truth, predictions))

                        precision    recall  f1-score   support

sehr wenig Information       0.87      0.52      0.65        25
      viel Information       0.57      0.50      0.53         8
     wenig Information       0.55      0.82      0.65        22

              accuracy                           0.64        55
             macro avg       0.66      0.61      0.61        55
          weighted avg       0.70      0.64      0.63        55

